In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np
import matplotlib.pyplot as plt

### Data Preparation

In [ ]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download the CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

In [ ]:
num_epochs = 10
num_classes = 10
batch_size = 64
learning_rate = 0.001

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle= True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle= False)

### Model Building, Training, Testing

In [ ]:
train_accuracy = []
test_accuracy = []

# 10 here is number of experiments. With this setup, you will have the training of 10 epochs for 10 times.
for i in range(10):

  # model
  class ConvNet(nn.Module):
      def __init__(self):
          super(ConvNet, self).__init__()
          self.layer1 = nn.Sequential(
              nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=2))

          self.layer2 = nn.Sequential(
              nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2, stride=2))

          self.drop_out = nn.Dropout(p = 0.5)   # ***************************************** You can change p value here *****************************************

          self.fc1 = nn.Linear(8*8*64, 100)
          self.act = nn.ReLU()
          self.fc2 = nn.Linear(100, 10)

      def forward(self, x):
          out = self.layer1(x)
          out = self.layer2(out)
          out = out.reshape(out.size(0), -1)
          out = self.drop_out(out)             # ************************************* Comment out this line to not use Dropout ***********************************
          out = self.fc1(out)
          out = self.act(out)
          out = self.fc2(out)
          return out


  model = ConvNet()

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.to(device)

  import torch
  from torchsummary import summary
  model.to('cuda')
  input_data = torch.randn(1, 3, 32, 32).to('cuda')
  summary(model, input_size=(3, 32, 32))

  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Train the model
  total_step = len(train_loader)
  loss_list = []
  acc_list = []

  loss_plot = []
  acc_plot = []
  epoch_plot = []

  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):

          images, labels = images.to(device), labels.to(device)

          outputs = model(images)
          loss = criterion(outputs, labels)
          loss_list.append(loss.item())

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          total = labels.size(0)
          _, predicted = torch.max(outputs.data, 1)
          correct = (predicted == labels).sum().item()
          acc_list.append(correct / total)

          epoch_plot.append(epoch+1)

          if (i + 1) % 50 == 0:
              print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                    .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                            (correct / total) * 100))
              loss_plot.append(loss.item())

              acc_plot.append(correct / total)

  train_accuracy.append((correct / total) * 100)

  from bokeh.io import output_notebook, show


  # Test the model
  model.eval()
  with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in test_loader:
          images, labels = images.to(device), labels.to(device)

          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

  test_accuracy.append((correct / total) * 100)


In [ ]:
print(train_accuracy)

In [ ]:
print(test_accuracy)